In [1]:
%pip install accelerate -U


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
dataset = pd.read_excel('dataset.xlsx')

In [4]:
import re

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Удаление ссылок
    text = re.sub(r'<.*?>', '', text)    # Удаление HTML-тегов
    text = re.sub(r'[^а-яА-Яa-zA-Z0-9\s]', '', text)  # Удаление специальных символов
    return text.strip()

# Применение функции
cleaned_data = [clean_text(example) for example in dataset]


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Alternate-Tokenizer")

def tokenize_text(text):
    return tokenizer.encode(text, truncation=True, max_length=512)

# Токенизация и нормализация данных
tokenized_data = [tokenize_text(example) for example in cleaned_data]


In [6]:
from sklearn.model_selection import train_test_split

# Разделение данных
train_data, temp_data = train_test_split(tokenized_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, train_size=0.5, test_size=0.5, random_state=42)


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./finetuned-llama',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    evaluation_strategy='steps',
    eval_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
)


/home/lockr/.local/share/virtualenvs/GBK_internship-EmzpK5a_/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
from transformers import Trainer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("google/flan-t5-base")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Запуск обучения
trainer.train()


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, Gemma2Config, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, JambaConfig, JetMoeConfig, LlamaConfig, MambaConfig, Mamba2Config, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MusicgenMelodyConfig, MvpConfig, NemotronConfig, OlmoConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, Phi3Config, PLBartCo

In [ ]:
import subprocess
import os

def fine_tune_ollama(model_name, dataset_path, output_dir, epochs=3, batch_size=8):
    # Создаем команду для обучения
    dataset_path = 
    train_command = [
        "ollama",
        "train",
        "--model", model_name,
        "--data", dataset_path,
        "--output", output_dir,
        "--epochs", str(epochs),
        "--batch-size", str(batch_size)
    ]

    # Запускаем команду
    try:
        result = subprocess.run(train_command, capture_output=True, text=True, check=True)
        print("Fine-tuning completed successfully.")
        print("Output:", result.stdout)
    except subprocess.CalledProcessError as e:
        print("An error occurred during fine-tuning.")
        print("Error:", e.stderr)

if __name__ == "__main__":
    # Параметры для дообучения
    model_name = "bambucha/saiga-llama3"  # Пример имени модели Ollama
    dataset_path = "dataset.xlsx"  # Путь к вашему датасету
    output_dir = "./fine-tuned-model"  # Директория для сохранения модели

    # Дообучение модели
    fine_tune_ollama(model_name, dataset_path, output_dir)
